In [1]:
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
import os
os.chdir('/Users/z00193k/Desktop/Projects/Project_2016/Intelligent_Tutoring_System')

In [2]:
!pwd

/Users/z00193k/Desktop/Projects/Project_2016/Intelligent_Tutoring_System


In [69]:
data_train = pd.read_csv('algebra_2008_2009/algebra_2008_2009_train.txt', delimiter = '\t', header = 0,
                         dtype = {'RowNo' : int, 'AnonStudentId' : str,'ProblemHierarchy' : str,'ProblemName' : str,'ProblemView' : int,'StepName' : str,'StepStartTime' : str,'FirstTransactionTime' : str,'CorrectTransactionTime' : str,'StepEndTime' : str,'StepDuration_sec' : int,'CorrectStepDuration_sec' : int,'ErrorStepDuration_sec' : int,'CorrectFirstAttempt' : int,'Incorrects' : int,'Hints' : int,'Corrects' : int,'KC_Default' : str,'Opportunity_Default' : str})

In [70]:
#data_train

In [71]:
data_train.replace('', np.nan)
data_train.replace('NULL', np.nan)
data_train['Anon Student Id'].fillna('DS_NULL_AnonStudentId', inplace=True)
data_train['Problem Hierarchy'].fillna('DS_NULL_ProblemHierarchy', inplace=True)
data_train['Problem View'].fillna('DS_NULL_ProblemView', inplace=True)
data_train['Problem Name'].fillna('DS_NULL_ProblemName', inplace=True)
data_train['Correct Step Duration (sec)'].fillna('DS_NULL_CorrectStepDur', inplace=True)
data_train['Error Step Duration (sec)'].fillna('DS_NULL_ErrorStepDur', inplace=True)
data_train['First Transaction Time'].fillna('DS_NULL_FirstTranTime', inplace=True)
data_train['Correct Transaction Time'].fillna('DS_NULL_CorrectTranTime', inplace=True)
data_train['Step End Time'].fillna('DS_NULL_StepEndTime', inplace=True)
data_train['Correct First Attempt'].fillna('DS_NULL_Correct_FA', inplace=True)
data_train['Incorrects'].fillna('DS_NULL_Incorrects', inplace=True)
data_train['Hints'].fillna('DS_NULL_Hints', inplace=True)
data_train['Corrects'].fillna('DS_NULL_Corrects', inplace=True)
data_train['Opportunity(SubSkills)'].fillna('DS_NULL_OPPO_SubSkills', inplace=True)
data_train['Step Name'].fillna('DS_NULL_StepName', inplace=True)
data_train['KC(SubSkills)'].fillna('DS_NULL_KC_SubSkills', inplace=True)
data_train['KC(KTracedSkills)'].fillna('DS_NULL_KC_KTracedSkills', inplace=True)
data_train['Opportunity(KTracedSkills)'].fillna('DS_NULL_OPPO_KTracedSkill', inplace=True)
data_train['KC(Rules)'].fillna('DS_NULL_KC_Rules', inplace=True)
data_train['Opportunity(Rules)'].fillna('DS_NULL_Opp_Rules', inplace=True)

#data_train.ix[:1000, :23]

#############################

In [72]:
import re
data_train=data_train


def word_list(word_list):
    splitline=[]
    word_inter = re.sub('[^a-zA-Z \n\.]', ' ', word_list)  
    splitline = word_inter.split(' ')
    #return splitline
    splitline_unique =[]
    for wordlst in splitline:
        if wordlst not in splitline_unique:
            if wordlst not in '':
                #wordlst="'"+wordlst+"'"
                splitline_unique.append(wordlst)
    return splitline_unique

def get_PN_Cluster_func(row):
    word_list_PN=[]
    splitline=[]
    word_inter = re.sub('[^a-zA-Z \n\.]', ' ', row)  
    splitline = word_inter.split(' ')
    splitline_unique =[]
    for wordlst in splitline:
        if wordlst not in splitline_unique:
            if wordlst not in '':
                #wordlst="'"+wordlst+"'"
                splitline_unique.append(wordlst)
    return splitline_unique[0]
    



#data_train_test["KCTracked_words"]=data_train_test.apply(lambda row: Clean_row("KC(KTracedSkills)"),axis=1)
#data_train_test["KCTracked_words"] = data_train["KC(KTracedSkills)"].map(lambda word: re.sub('[^a-zA-Z \n\.]', ' ', word))

data_train["KCTracked_words"] = data_train["KC(KTracedSkills)"].map(lambda word: word_list(word))
data_train["KCRules_words"] = data_train["KC(Rules)"].map(lambda word: word_list(word))
data_train["PN_Clust"] = data_train["Problem Name"].map(lambda row: get_PN_Cluster_func(row))

data_train["KCTracked_words_Hash"]=data_train["KCTracked_words"].apply(lambda x : tuple(x) if type(x) is list else x)
data_train["KCRules_words_Hash"]=data_train["KCRules_words"].apply(lambda x : tuple(x) if type(x) is list else x)

In [73]:
#data_train

##################################

In [74]:
len(data_train["PN_Clust"].unique())

119

In [75]:

import re
def unique_list(col):
    word_list=[]
    splitline=[]
    #splitline_unique =[]
    for word in col:
        word_inter = re.sub('[^a-zA-Z \n\.]', ' ', word)  
        splitline = word_inter.split(' ')
        splitline_unique =[]
        for wordlst in splitline:
            if wordlst not in splitline_unique:
                if wordlst not in '':
                    splitline_unique.append(wordlst)
            
        word_list.append(splitline_unique)
    return word_list
    
#print word_list_KCT

KCT_unique = data_train["KC(KTracedSkills)"].unique()
KCR_unique = data_train["KC(Rules)"].unique()

In [76]:
word_list_KCT = unique_list(KCT_unique)
word_list_KCR = unique_list(KCR_unique)

##### Getting the cosine similarity between the sentences

In [77]:

def creating_clust(col_list):
    count=0
    lst_iter = col_list
    cluster = 1
    dic={}
    for lst1 in col_list:
        lst_iter.remove(lst1)
    
        list_clust=[]
        list_clust.append(lst1)
        for lst2 in lst_iter:
            word_match = len(list(set(lst1).intersection(lst2)))
            pct_match = (word_match*100)/len(lst1)
            if pct_match>70:
                lst_iter.remove(lst2)
                list_clust.append(lst2)
                #print lst1, lst2, pct_match, cluster
    
        dic["Cluster"+str(cluster)]=list_clust
        count=count+1
        cluster=cluster+1
        #lst_initial=lst_inter
        #print lst_initial
    return dic


In [78]:
cluster_KCT=creating_clust(word_list_KCT)
cluster_KCR=creating_clust(word_list_KCR)

cluster_KCT_df = pd.DataFrame.from_dict(cluster_KCT, orient='index')
cluster_KCT_df.reset_index(level=0, inplace=True)

cluster_KCR_df = pd.DataFrame.from_dict(cluster_KCR, orient='index')
cluster_KCR_df.reset_index(level=0, inplace=True)

In [79]:
def transposing(data):

    data.set_index('index',inplace=True)
    data.reset_index(inplace=True)
    Clusters = data["index"].tolist()

    i=0

    for Clustername in Clusters:
        temp = data[data["index"]==Clustername].T
        temp['ClusterNum']= Clustername
        temp = temp.ix[1:]
        temp.columns = ['Cols', 'ClusterNum']
    
        if(i==0):
            Clusterframe = temp
            i+=1
        else:
            Clusterframe = Clusterframe.append(temp)
        
    Clusterframe.reset_index(inplace=True)        
    Clusterframe.drop('index',1,inplace=True)
    
    return Clusterframe

In [80]:
cluster_KCT_df_T=transposing(cluster_KCT_df)
cluster_KCT_df_T['col_tup'] = cluster_KCT_df_T['Cols'].apply(lambda x : tuple(x) if type(x) is list else x)
cluster_KCT_df_T.drop_duplicates(subset=['col_tup', 'ClusterNum'])


cluster_KCR_df_T=transposing(cluster_KCR_df)
cluster_KCR_df_T['col_tup'] = cluster_KCR_df_T['Cols'].apply(lambda x : tuple(x) if type(x) is list else x)
cluster_KCR_df_T.drop_duplicates(subset=['col_tup', 'ClusterNum'])

,Cols,ClusterNum,col_tup
0,"[SET, MIN, BOUND, HIGH]",Cluster10,"(SET, MIN, BOUND, HIGH)"
1,"[SET, MAX, BOUND, HIGH]",Cluster10,"(SET, MAX, BOUND, HIGH)"
2,"[SET, MIN, BOUND, SLOPE, INTERCEPT, HIGH]",Cluster10,"(SET, MIN, BOUND, SLOPE, INTERCEPT, HIGH)"
6,"[SET, MIN, BOUND, SLOPE, INTERCEPT, LOW]",Cluster10,"(SET, MIN, BOUND, SLOPE, INTERCEPT, LOW)"
7,"[SET, MIN, BOUND, HIGH, SLOPE, INTERCEPT, LOW]",Cluster10,"(SET, MIN, BOUND, HIGH, SLOPE, INTERCEPT, LOW)"
8,"[SET, MAX, BOUND, HIGH, LOW]",Cluster10,"(SET, MAX, BOUND, HIGH, LOW)"
9,"[unknown, bug, element, SET, MIN, BOUND, HIGH]",Cluster10,"(unknown, bug, element, SET, MIN, BOUND, HIGH)"
10,"[SET, MIN, BOUND, SLOPE, INTERCEPT, HIGH, LOW]",Cluster10,"(SET, MIN, BOUND, SLOPE, INTERCEPT, HIGH, LOW)"
13,"[SET, MAX, BOUND, LOW, HIGH]",Cluster10,"(SET, MAX, BOUND, LOW, HIGH)"
15,"[SET, MIN, BOUND]",Cluster10,"(SET, MIN, BOUND)"


In [81]:
pd.merge(data_train, cluster_KCT_df_T, left_on='KCTracked_words_Hash', right_on = 'col_tup', how='left', suffixes=('_A', '_B'))

pd.merge(data_train, cluster_KCR_df_T, left_on='KCRules_words_Hash', right_on = 'col_tup', how='left', suffixes=('_A', '_C'))
    

,Row,Anon Student Id,Problem Hierarchy,Problem Name,Problem View,Step Name,Step Start Time,First Transaction Time,Correct Transaction Time,Step End Time,...,KC(Rules),Opportunity(Rules),KCTracked_words,KCRules_words,PN_Clust,KCTracked_words_Hash,KCRules_words_Hash,Cols,ClusterNum,col_tup
0,1,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R2C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,UNIT-HELP,1,"[DS, NULL, KC, KTracedSkills]","[UNIT, HELP]",REAL,"(DS, NULL, KC, KTracedSkills)","(UNIT, HELP)","[UNIT, HELP]",Cluster1,"(UNIT, HELP)"
1,1,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R2C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,UNIT-HELP,1,"[DS, NULL, KC, KTracedSkills]","[UNIT, HELP]",REAL,"(DS, NULL, KC, KTracedSkills)","(UNIT, HELP)","[UNIT, HELP]",Cluster1,"(UNIT, HELP)"
2,2,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R3C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,VARIABLE-HELP,1,"[DS, NULL, KC, KTracedSkills]","[VARIABLE, HELP]",REAL,"(DS, NULL, KC, KTracedSkills)","(VARIABLE, HELP)",NaN,NaN,NaN
3,3,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R3C2,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,STANDARD-MX+B-FORMULA-HELP,1,"[Using, simple, numbers, large, Write, express...","[STANDARD, MX, B, FORMULA, HELP]",REAL,"(Using, simple, numbers, large, Write, express...","(STANDARD, MX, B, FORMULA, HELP)","[STANDARD, MX, B, FORMULA, HELP]",Cluster2,"(STANDARD, MX, B, FORMULA, HELP)"
4,3,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R3C2,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,STANDARD-MX+B-FORMULA-HELP,1,"[Using, simple, numbers, large, Write, express...","[STANDARD, MX, B, FORMULA, HELP]",REAL,"(Using, simple, numbers, large, Write, express...","(STANDARD, MX, B, FORMULA, HELP)","[STANDARD, MX, B, FORMULA, HELP]",Cluster2,"(STANDARD, MX, B, FORMULA, HELP)"
5,4,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R4C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,GIVEN-HELP-NON-NUMERIC-PHRASE,1,"[Entering, a, given]","[GIVEN, HELP, NON, NUMERIC, PHRASE]",REAL,"(Entering, a, given)","(GIVEN, HELP, NON, NUMERIC, PHRASE)","[GIVEN, HELP, NON, NUMERIC, PHRASE]",Cluster264,"(GIVEN, HELP, NON, NUMERIC, PHRASE)"
6,4,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R4C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,GIVEN-HELP-NON-NUMERIC-PHRASE,1,"[Entering, a, given]","[GIVEN, HELP, NON, NUMERIC, PHRASE]",REAL,"(Entering, a, given)","(GIVEN, HELP, NON, NUMERIC, PHRASE)","[GIVEN, HELP, NON, NUMERIC, PHRASE]",Cluster264,"(GIVEN, HELP, NON, NUMERIC, PHRASE)"
7,5,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R4C2,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,CALCULATED-VALUE-HELP-MX+B-GIVEN-X-ZERO,1,"[Using, simple, numbers, large, Find, Y, negat...","[CALCULATED, VALUE, HELP, MX, B, GIVEN, X, ZERO]",REAL,"(Using, simple, numbers, large, Find, Y, negat...","(CALCULATED, VALUE, HELP, MX, B, GIVEN, X, ZERO)","[CALCULATED, VALUE, HELP, MX, B, GIVEN, X, ZERO]",Cluster3,"(CALCULATED, VALUE, HELP, MX, B, GIVEN, X, ZERO)"
8,6,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R5C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,GIVEN-HELP-NONE,1,"[Entering, a, given]","[GIVEN, HELP, NONE]",REAL,"(Entering, a, given)","(GIVEN, HELP, NONE)","[GIVEN, HELP, NONE]",Cluster239,"(GIVEN, HELP, NONE)"
9,6,stu_de2777346f,"Unit CTA1_01, Section CTA1_01-3",REAL20B,1,R5C1,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,2008-09-19 13:30:46.0,...,GIVEN-HELP-NONE,1,"[Entering, a, given]","[GIVEN, HELP, NONE]",REAL,"(Entering